In [1]:
import os
import sys
from pathlib import Path
# Get the current working directory
cwd = Path(os.getcwd())
# Assuming your notebook is in the project root, set the project root as cwd
project_root = cwd.parent
print(project_root)

# If your notebook is in a subdirectory of the project root, you can modify the path accordingly:
# project_root = cwd.parent  # Go up one directory level
# project_root = cwd.parent.parent  # Go up two directory levels
# Add the project root directory to sys.path
sys.path.insert(0, str(project_root))
# sys.path.append('../')

import xpt.fit_analysis as xfa
import warnings
import corr_fitter.bs_utils as bs
import corr_fitter.bs_analysis as bs_analysis
import corr_fitter.load_data_priors as ld
import corr_fitter.corr_fit_analysis as fa
import xpt.priors as priors
import xpt.i_o as i_o
import xpt.fit_routine as fit
import xpt.plots as plots
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib
import tqdm
import h5py as h5
import yaml
import io
import lsqfit
import importlib
import numpy as np
import gvar as gv
import pandas as pd
# from h5glance import H5Glance


warnings.simplefilter(action="default")
warnings.filterwarnings('ignore')


/home/gmoney/lqcd/hyperon-xpt-master


In [2]:
from importlib import reload
from tabulate import tabulate
importlib.reload(xfa)
importlib.reload(i_o)
importlib.reload(fit)
importlib.reload(priors)

<module 'xpt.priors' from '/home/gmoney/lqcd/hyperon-xpt-master/xpt/priors.py'>

In [3]:
# Define paths and other variables
base_dir = "/home/gmoney/lqcd/hyperon-xpt-master"
input_dir = os.path.join(base_dir, "tests/input_files")
data_dir = os.path.join(base_dir, "data")
fit_results_dir = os.path.join(base_dir, "fit_results")
data_file = os.path.join(data_dir, "c51_2pt_octet_decuplet.h5")
hyperon_data_file = os.path.join(data_dir,"hyperon_data.h5")
bs_data_file = os.path.join(data_dir, "hyperon_bs_data.h5")
model_type = "all"
t_plot_min = 0
t_plot_max = 40
bs_N = 100
bs_seed = "seed"

In [4]:
importlib.reload(i_o)
input_output = i_o.InputOutput()
data = input_output.get_data()
print(data)
scale_setting_data = input_output.get_scale_setting_data()
# print(scale_setting_data)
# hyperon_data = ld.get_hyperon_posterior(bs_data=scale_setting_data)

# data,ensembles = input_output.get_data(scheme='w0_imp',units='Fpi')
# data_div, ensembles = input_output.get_data(scheme='w0_imp',units='Fpi')
# prior = input_output.make_prior(data,priors)
# phys_point_data = input_output.get_data_phys_point()


['a06m310L', 'a09m135', 'a09m350', 'a09m400', 'a12m130', 'a12m220L', 'a12m220S', 'a12m310', 'a12m350', 'a12m400', 'a15m220', 'a15m310']


KeyError: 'm_lambda'

In [5]:

new_prior = priors.get_prior(units='lam_chi')
data, ens = input_output.get_data(units='lam_chi')
print(data)
# new_prior = input_output.make_prior(data,prior)
phys_point_data = input_output.get_data_phys_point()

# initialize xpt models
with open('../xpt/models.yaml', 'r') as f:
    models = yaml.load(f, Loader=yaml.FullLoader)

xi_models = models['models']['xi_system']

results = []
# for key, item in xi_models.items():
model_fit = fit.FitRoutine(
data=data, prior=new_prior, emp_bayes=False, empbayes_grouping=None,
model_info=xi_models['xi:xi_st:s_n2lo:d_n2lo:x_n2lo:l_n2lo:w0imp:fv:xpt'])
my_fit = model_fit.fit
print(my_fit)
if my_fit is None:
    print(f"Skipping model {key} due to fit error.")

    # # Append results to list
    # try:
    #     result = {'Model': key, 'Chi2': my_fit_recalibrated.chi2, 'q': my_fit_recalibrated.Q}
    #     if 'm_{xi,0}' in my_fit_recalibrated.p:
    #         result['xi'] = my_fit_recalibrated.p['m_{xi,0}']
    #     if 'm_{xi_st,0}' in my_fit_recalibrated.p:
    #         result['xi_st'] = my_fit_recalibrated.p['m_{xi_st,0}']
    #     results.append(result)
    # except KeyError as e:
    #     print(f"KeyError encountered for model {key}: {e}")
    #     continue

# # create dataframe from results list and print it
# results_df = pd.DataFrame(results)
# print(tabulate(results_df, headers='keys', tablefmt='psql'))

# # plot results
# fig, ax = plt.subplots()
# ax.scatter(results_df['q'], results_df['Chi2'], marker='o')
# ax.set_xlabel('q')
# ax.set_ylabel('Chi2')
# ax.set_title('xi system')
# for i, txt in enumerate(results_df['Model']):
#     ax.annotate(txt, (results_df['q'][i], results_df['Chi2'][i]))
# plt.show()


{'m_lambda': array([0.95044(11), 0.952345(59), 0.948499(74), 0.66708(10), 0.928243(97),
       0.94445(10), 0.939460(65), 0.937488(91), 0.939525(63),
       0.919493(55), 0.934448(82)], dtype=object), 'm_sigma': array([0.99307(12), 0.991715(86), 0.980024(75), 0.725263(55),
       0.998798(70), 1.008179(97), 0.987352(70), 0.985272(69),
       0.973498(66), 0.995677(75), 0.994471(39)], dtype=object), 'm_sigma_st': array([1.13706(43), 1.18745(29), 1.16491(17), 0.89358(12), 1.223188(94),
       1.23284(14), 1.19129(23), 1.18605(19), 1.16723(21), 1.20257(41),
       1.205044(54)], dtype=object), 'm_xi_st': array([1.24413(20), 1.22834(47), 1.21621(11), 0.938567(83), 1.285457(79),
       1.29674(10), 1.25032(13), 1.23455(15), 1.21659(11), 1.26483(15),
       1.251981(44)], dtype=object), 'm_xi': array([1.068067(91), 1.049121(58), 1.027223(53), 0.777457(34),
       1.059837(88), 1.072528(88), 1.043192(60), 1.031971(74),
       1.018479(57), 1.048338(39), 1.040688(37)], dtype=object), 'm_omega'

TypeError: FitRoutine.__init__() missing 1 required positional argument: 'phys_point_data'

## Xi, Xi* system SU(2) HBXPT extrapolation ##

In [ ]:
# empirical bayes prior study 
importlib.reload(fit)
importlib.reload(priors)
print(np.log(5))
emp_fit = fit.FitRoutine(empbayes_grouping='disc_only',emp_bayes=True,
data=data, prior=new_prior, model_info=xi_models['xi:xi_st:s_n2lo:d_n2lo:x_n2lo:l_n2lo:w0imp:fv:xpt'])
# emp = emp_fit._make_empbayes_fit()
# print(emp)/

1.6094379124341003


TypeError: FitRoutine.__init__() missing 1 required positional argument: 'phys_point_data'

In [7]:
lam_sigma_models = models['models']['lam_sigma_system']
# importlib.reload(priors)
new_prior = priors.get_prior(units='lam_chi')
print(new_prior)
# for key, item in xi_models.items():
model_fit = fit.FitRoutine(phys_point_data=phys_point_data,
data=data, prior=new_prior, emp_bayes=False, empbayes_grouping=None,
model_info=lam_sigma_models['lam:sigma:sigma_st:s_n2lo:d_n2lo:x_n2lo:l_n2lo'])
my_fit = model_fit.fit
print(my_fit)

{'m_{xi,0}': 1.0(1.0), 'm_{xi_st,0}': 1.0(1.0), 'm_{lambda,0}': 1.0(1.0), 'm_{sigma,0}': 1.0(1.0), 'm_{sigma_st,0}': 1.0(1.0), 'm_{omega,0}': 1.0(1.0), 's_{xi}': 0(10), 's_{xi,bar}': 0(10), 's_{lambda}': 0.0(5.0), 's_{sigma,bar}': 0.0(5.0), 's_{sigma}': 0.0(5.0), 's_{omega,bar}': 0.0(5.0), 'g_{xi,xi}': 0.3(3.0), 'g_{xi_st,xi}': 0.7(3.0), 'g_{xi_st,xi_st}': -0.5(3.0), 'g_{lambda,sigma}': 0.0(5.0), 'g_{lambda,sigma_st}': 0.0(5.0), 'g_{sigma,sigma}': 0.0(5.0), 'g_{sigma_st,sigma}': 0.0(5.0), 'g_{sigma_st,sigma_st}': 0.0(5.0), 'a_{xi,4}': 0.0(5.0), 'a_{xi_st,4}': 0.0(5.0), 'b_{xi,4}': 0.0(5.0), 'b_{xi_st,4}': 0.0(5.0), 'a_{sigma,4}': 0.0(5.0), 'a_{sigma_st,4}': 0.0(5.0), 'b_{sigma,4}': 0.0(5.0), 'b_{sigma_st,4}': 0.0(5.0), 'a_{lambda,4}': 0.0(5.0), 'b_{lambda,4}': 0.0(5.0), 'a_{omega,4}': 0.0(5.0), 'b_{omega,4}': 0.0(5.0), 'a_{omega,6}': 0.0(5.0), 'b_{omega,6}': 0.0(5.0), 'd_{xi,a}': 0(10), 'd_{xi_st,a}': 0(10), 'd_{xi,s}': 0(10), 'd_{xi_st,s}': 0(10), 'd_{lambda,s}': 0(10), 'd_{lambda,a}'

ZeroDivisionError: float division

In [ ]:

import xpt.plots as plots
importlib.reload(plots)
plots.plot_fit(ens=ens,fit_out=my_fit,xparam='xi',)

KeyError: 'undefined key: xi'

In [ ]:
['fit_div_lam = fit.fit_routine(prior=prior, data=data_div, model_info=model_info_div_lam)
fit_div_lam_out = fit_div_lam.fit
print(fit_div_lam_out)


SyntaxError: unterminated string literal (detected at line 1) (346027678.py, line 1)

In [ ]:
fit_sigma = fit.fit_routine(prior=prior, data=data,
                            model_info=model_info_sigma)
fit_sigma_out = fit_sigma.fit
print(fit_sigma_out.y)


{
    'proton':
    array([1121(10), 958.4(7.7), 984.4(7.9), 1110.7(8.9), 1158.1(9.3),
           1211.3(9.7), 934.1(6.8), 998.3(7.3), 1074.4(7.8), 1124.0(8.2),
           1168.4(8.5), 950.8(6.3), 993.1(6.6), 1078.1(7.2), 1057.1(7.0),
           1092.9(7.3), 1158.1(7.7)], dtype=object),
}


In [ ]:
fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)
fit_out = fit_test.fit
print(fit_out.format(maxline=True))


Least Square Fit:
  chi2/dof [dof] = 1.3 [17]    Q = 0.17    logGBF = -85.26

Parameters:
     m_{proton,0}        902 (10)      [     900 (400) ]  
     b_{proton,2}     -6e-13 +- 5      [     0.0 (5.0) ]  
     b_{proton,4}     -4e-12 +- 5      [     0.0 (5.0) ]  
     b_{proton,6}       0.4 (5.0)      [     0.0 (5.0) ]  
     d_{proton,a}     -4e-13 +- 5      [     0.0 (5.0) ]  
    d_{proton,aa}      2e-12 +- 5      [     0.0 (5.0) ]  
    d_{proton,al}     -2e-12 +- 5      [     0.0 (5.0) ]  
   d_{proton,all}      -1.4 (4.9)      [     0.0 (5.0) ]  
   d_{proton,aal}       0.2 (1.0)      [     0.0 (5.0) ]  
     d_{proton,s}      -0.75 (74)      [     0.0 (5.0) ]  
    d_{proton,as}       8.7 (2.1)      [     0.0 (5.0) ]  *
    d_{proton,ls}      -2.5 (4.8)      [     0.0 (5.0) ]  
    d_{proton,ss}       2.8 (4.4)      [     0.0 (5.0) ]  
g_{proton,proton}     0.02 (4.99)      [     1.3 (5.0) ]  
 g_{proton,delta}      -1.13 (41)      [     0.9 (5.0) ]  
      m_{delta,0}      1

In [ ]:
from importlib import reload
reload(fa)
fit_analysis = fa.fit_analysis(
    phys_point_data=phys_point_data, data=data, model_info=model_info, prior=prior)
fit_analysis_sigma = fa.fit_analysis(
    phys_point_data=phys_point_data, data=data, model_info=model_info_sigma, prior=prior)
extrap_mass = fit_analysis.extrapolated_mass
extrap_mass_deriv = fit_analysis_sigma.extrapolated_mass
m_proton = extrap_mass['proton']
m_proton_deriv = extrap_mass_deriv['proton']


{'proton': 944.0(7.1)}


In [ ]:
reload(io)
m_pi = phys_point_data['m_pi']
lam_chi = phys_point_data['lam_chi']
e_pi = m_pi / lam_chi
sigma = m_pi**2 * gv.deriv(m_proton, m_pi**2)
l3 = gv.gvar('3.53(26)')
l4 = gv.gvar('4.73(10)')
sigma_analytic = e_pi/2 * (1 + e_pi**2*(5/2 - l3/2 - 2*l4)) * m_proton_deriv
print('sigma w/ gv.deriv:', sigma)
print('sigma analytic:', sigma_analytic)


sigma w/ gv.deriv: 53.12(24)
sigma analytic: 48.53(38)


In [ ]:
print(fit_analysis.error_budget)


KeyError: 'm_{proton,0}'